## Day 10: Cathode-Ray Tube

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day10/puzzle.ipynb)

In [27]:
#!value --name sampleRaw --from-file ./data_sample.txt


In [28]:
#!value --name actualRaw --from-file ./data_actual.txt

In [29]:
type Op = 
| NoOp
| AddX of int

In [30]:
#load "../common/common.fsx"

let parse input = 
    Pattern1.read (
        function 
        | Regex "noop" [] -> NoOp 
        | Regex "addx ([-0-9]+)" [num] -> num |> int |> AddX
        | _ -> failwith "unknown command"
    ) input

In [31]:
#!share sampleRaw --from value
#!share actualRaw --from value

let sampleCommands = parse sampleRaw
let actualCommands = parse actualRaw

sampleCommands |> Seq.take 10 |> display

index,type,Item
0,FSI_0036+Op+AddX,15
1,FSI_0036+Op+AddX,-11
2,FSI_0036+Op+AddX,6
3,FSI_0036+Op+AddX,-3
4,FSI_0036+Op+AddX,5
5,FSI_0036+Op+AddX,-1
6,FSI_0036+Op+AddX,-8
7,FSI_0036+Op+AddX,13
8,FSI_0036+Op+AddX,4
9,FSI_0036+Op+_NoOp,


In [32]:
actualCommands |> Seq.take 10 |> display

index,type,Item
0,FSI_0036+Op+_NoOp,
1,FSI_0036+Op+AddX,5
2,FSI_0036+Op+_NoOp,
3,FSI_0036+Op+AddX,3
4,FSI_0036+Op+AddX,-2
5,FSI_0036+Op+AddX,4
6,FSI_0036+Op+_NoOp,
7,FSI_0036+Op+_NoOp,
8,FSI_0036+Op+_NoOp,
9,FSI_0036+Op+_NoOp,


### Part 1

In [33]:
module Part1 = 
    type State = { Cycle : int; X: int; SignalAcc : int}
    with static member Empty = { Cycle= 1; X = 1; SignalAcc = 0 }


The instructions are not elementary. We can iteprete and implement them as sequences of more little ones:
1. Cycle increment
2. X register increase
3. Signal strength computation and accumulation

In [34]:
module Part1 = 
    let cycle (state : Part1.State) = 
        { state with Cycle = state.Cycle + 1 }
    let increase addend (state : Part1.State) = 
        { state with X = state.X + addend }
    let sampleSignalStrength (state : Part1.State) = 
        if ((state.Cycle - 20) % 40 = 0) 
        then { state with SignalAcc = state.SignalAcc + (state.X * state.Cycle) }
        else state

    let execute state op = 
        match op with
        | NoOp -> 
            state 
            |> sampleSignalStrength 
            |> cycle
        | AddX x -> 
            state 
            |> sampleSignalStrength 
            |> cycle
            |> sampleSignalStrength 
            |> cycle
            |> increase x


In [35]:
let allSampleStates =
    sampleCommands
    |> Array.mapFold (
        fun state op ->
            let newState = Part1.execute state op
            newState,newState
    ) Part1.State.Empty
    |> fst 
allSampleStates
|> Array.filter (fun state -> (state.Cycle - 20) % 40 = 0)
|> display

Array.last allSampleStates


index,Cycle,X,SignalAcc
0,20,21,0
1,60,19,420
2,100,18,1560
3,140,21,3360
4,180,16,6300


Cycle,X,SignalAcc
241,17,13140


In [36]:
actualCommands 
|> Array.fold Part1.execute Part1.State.Empty

Cycle,X,SignalAcc
241,37,15120


### Part 2

For Part 2, the same approach is applicable.

In [37]:
module Part2 = 
    type Pixel = 
    | Lit
    | Dark

    type State = { X: int; Sprite: int array; TubePosition: int*int; Pixels: Pixel[,] }
    with static member Empty = {
            X = 1
            Sprite = [| 0;1;2 |]
            TubePosition = 0,0
            Pixels = Array2D.create 6 40 Dark
        }

    let cycle (state : State) =
        let (hPos, vPos) = state.TubePosition
        let vCarry, hReminder = Math.DivRem(hPos + 1, 40)
        { state with TubePosition = hReminder, (vPos + vCarry)%6 }
    let increase addend (state : State) = 
        let newX = state.X + addend
        { state with
            X = newX
            Sprite = [| newX - 1; newX; newX + 1 |] }
    let drawPixel (state : State) = 
        let (hPos, vPos) = state.TubePosition
        if state.Sprite |> Array.contains hPos
        then 
            let newPixels = 
                state.Pixels
                |> Array2D.copy
            Array2D.set newPixels vPos hPos Lit
            { state with Pixels = newPixels }
        else state

    let execute state op = 
        match op with
        | NoOp -> 
            state 
            |> drawPixel
            |> cycle
        | AddX x -> 
            state 
            |> drawPixel
            |> cycle
            |> drawPixel
            |> cycle
            |> increase x

In [38]:
#load "../common/matrixFormatting.fsx"
open System.Drawing

(sampleCommands 
|> Array.fold Part2.execute Part2.State.Empty
).Pixels
|> toDisplayable (function | Part2.Dark -> Color.Transparent | Part2.Lit -> Color.Green)
|> withSettings { Width = 400 }

In [39]:

open System.Drawing


(actualCommands 
|> Array.fold Part2.execute Part2.State.Empty
).Pixels
|> toDisplayable (function | Part2.Dark -> Color.Transparent | Part2.Lit -> Color.Green)
|> withSettings { Width = 400 }